In [3]:
import pandas as pd
import os

In [4]:
df = pd.read_csv(os.path.join(os.path.abspath('../data'), 'houses_to_rent_v2_etl.csv'))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9347 entries, 0 to 9346
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 9347 non-null   object
 1   area                 9347 non-null   int64 
 2   rooms                9347 non-null   int64 
 3   bathroom             9347 non-null   int64 
 4   parking spaces       9347 non-null   int64 
 5   floor                9347 non-null   int64 
 6   animal               9347 non-null   object
 7   furniture            9347 non-null   object
 8   hoa (R$)             9347 non-null   int64 
 9   rent amount (R$)     9347 non-null   int64 
 10  property tax (R$)    9347 non-null   int64 
 11  fire insurance (R$)  9347 non-null   int64 
 12  total (R$)           9347 non-null   int64 
dtypes: int64(10), object(3)
memory usage: 949.4+ KB


In [6]:
df.describe()

,area,rooms,bathroom,parking spaces,floor,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
count,9347.000000,9347.000000,9347.000000,9347.000000,9347.000000,9347.000000,9347.000000,9347.000000,9347.000000,9347.000000
mean,114.102921,2.330373,1.978817,1.357655,6.504868,723.533754,3274.745694,192.802076,44.542206,4235.803466
std,96.509641,1.074270,1.187702,1.362554,4.573303,753.693087,2727.978768,243.805761,37.946131,3307.270778
min,11.000000,1.000000,1.000000,0.000000,1.000000,0.000000,450.000000,0.000000,3.000000,499.000000
25%,52.000000,1.000000,1.000000,0.000000,3.000000,170.000000,1450.000000,28.000000,20.000000,1926.000000
50%,80.000000,2.000000,2.000000,1.000000,7.000000,500.000000,2390.000000,100.000000,32.000000,3146.000000
75%,142.000000,3.000000,3.000000,2.000000,8.000000,1000.000000,4000.000000,252.000000,54.000000,5363.500000
max,2000.000000,10.000000,10.000000,10.000000,51.000000,4000.000000,30000.000000,1193.000000,451.000000,31010.000000


### Duplicate Values

We have seen in our data exploration that there are some duplicated values. Let's check.

In [7]:
df.duplicated().sum()

352

In [8]:
df[df.duplicated()]

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
95,Belo Horizonte,40,1,1,1,2,not acept,furnished,0,970,0,13,983
201,Belo Horizonte,306,3,5,0,7,acept,not furnished,0,8000,0,132,8132
233,Belo Horizonte,64,3,1,0,3,not acept,not furnished,215,1400,27,19,1661
275,Belo Horizonte,28,1,1,0,7,not acept,furnished,550,1100,0,15,1665
291,Belo Horizonte,59,2,2,0,2,not acept,furnished,510,1400,112,19,2041
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261,São Paulo,50,2,3,0,7,not acept,not furnished,0,1740,84,27,1851
9273,São Paulo,113,2,1,1,5,acept,not furnished,1000,3000,50,39,4089
9277,São Paulo,20,1,1,0,2,acept,furnished,602,1800,130,23,2555
9290,São Paulo,90,2,2,1,3,acept,not furnished,1000,3000,50,39,4089


We don't have any field that exclusively identify the property. Since our model will not benefit from duplicate values, let's drop them and keep the first value.

In [9]:
df = df.drop_duplicates(keep='first')

In [10]:
df.shape

(8995, 13)

### Cross-field validation

Since our target it's the rent price of the property, let's check the integrity of the monetary columns related to rent amount.

Normally, these values can't be more expensive then the rent price, so let's check.

In [11]:
df.loc[df['property tax (R$)'] > df['rent amount (R$)']]

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)


In [12]:
df.loc[df['fire insurance (R$)'] > df['rent amount (R$)']]

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)


In [13]:
df.loc[df['hoa (R$)'] > df['rent amount (R$)']]

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
252,Belo Horizonte,21,1,1,1,2,not acept,not furnished,1000,980,0,14,1994
329,Belo Horizonte,85,2,2,1,6,acept,not furnished,1105,1100,119,15,2339
388,Belo Horizonte,57,1,1,1,6,not acept,not furnished,1500,1400,158,19,3077
408,Belo Horizonte,31,1,1,1,3,not acept,not furnished,550,460,92,7,1109
990,Belo Horizonte,102,3,2,2,6,acept,not furnished,1150,1100,0,15,2265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8861,São Paulo,299,3,3,3,1,acept,not furnished,2689,1942,316,25,4972
8924,São Paulo,30,1,1,1,7,not acept,not furnished,1959,1630,227,21,3837
8955,São Paulo,68,3,1,2,4,acept,not furnished,990,882,20,12,1904
9118,São Paulo,173,3,4,3,2,acept,not furnished,3000,1950,796,25,5771


We can see that only hoa column have values more expensive than the rent price.

To confirm that behavior, we did a little research and discovered that are possible that hoa it's more expensive then rent price, so in this case we will not drop these values.

- Data Types - ok
- Ranges - ok
- Emptiness - ok
- Uniqueness - ok
- Cross Field Validation - ok

In [14]:
df.to_csv(os.path.join(os.path.abspath('../data'), 'houses_to_rent_v2_dtcl.csv'), index=False)